In [1]:
import os
import glob
import pymongo
import numpy as np
import pandas as pd
import pickle
import datetime
import time
import gzip
import lzma
import pytz

### 1. snapshot

In [3]:
logSZ1 = pd.read_csv(r'\\192.168.10.30\Kevin_zhenyu\check\20200528\premdgw\mdL2Log_SZ.csv',
                    encoding="utf-8", nrows=1000000)
pd.set_option("max_columns", 200)
logSZ1[(logSZ1["ID"] == 2000533) & (logSZ1["time"] > 94000000)].head(10)

,clockAtArrival,sequenceNo,ID,time,preClose,open,high,low,close,cum_tradesCnt,cum_volume,cum_amount,vwapBid,vwapAsk,totalBidQuantity,totalAskQuantity,bid1p,bid2p,bid3p,bid4p,bid5p,bid6p,bid7p,bid8p,bid9p,bid10p,ask1p,ask2p,ask3p,ask4p,ask5p,ask6p,ask7p,ask8p,ask9p,ask10p,bid1q,bid2q,bid3q,bid4q,bid5q,bid6q,bid7q,bid8q,bid9q,bid10q,ask1q,ask2q,ask3q,ask4q,ask5q,ask6q,ask7q,ask8q,ask9q,ask10q,bid1n,bid2n,bid3n,bid4n,bid5n,bid6n,bid7n,bid8n,bid9n,bid10n,ask1n,ask2n,ask3n,ask4n,ask5n,ask6n,ask7n,ask8n,ask9n,ask10n,bid1Top1q,bid1Top2q,bid1Top3q,bid1Top4q,bid1Top5q,bid1Top6q,bid1Top7q,bid1Top8q,bid1Top9q,bid1Top10q,bid1Top11q,bid1Top12q,bid1Top13q,bid1Top14q,bid1Top15q,bid1Top16q,bid1Top17q,bid1Top18q,bid1Top19q,bid1Top20q,bid1Top21q,bid1Top22q,bid1Top23q,bid1Top24q,bid1Top25q,bid1Top26q,bid1Top27q,bid1Top28q,bid1Top29q,bid1Top30q,bid1Top31q,bid1Top32q,bid1Top33q,bid1Top34q,bid1Top35q,bid1Top36q,bid1Top37q,bid1Top38q,bid1Top39q,bid1Top40q,bid1Top41q,bid1Top42q,bid1Top43q,bid1Top44q,bid1Top45q,bid1Top46q,bid1Top47q,bid1Top48q,bid1Top49q,bid1Top50q,ask1Top1q,ask1Top2q,ask1Top3q,ask1Top4q,ask1Top5q,ask1Top6q,ask1Top7q,ask1Top8q,ask1Top9q,ask1Top10q,ask1Top11q,ask1Top12q,ask1Top13q,ask1Top14q,ask1Top15q,ask1Top16q,ask1Top17q,ask1Top18q,ask1Top19q,ask1Top20q,ask1Top21q,ask1Top22q,ask1Top23q,ask1Top24q,ask1Top25q,ask1Top26q,ask1Top27q,ask1Top28q,ask1Top29q,ask1Top30q,ask1Top31q,ask1Top32q,ask1Top33q,ask1Top34q,ask1Top35q,ask1Top36q,ask1Top37q,ask1Top38q,ask1Top39q,ask1Top40q,ask1Top41q,ask1Top42q,ask1Top43q,ask1Top44q,ask1Top45q,ask1Top46q,ask1Top47q,ask1Top48q,ask1Top49q,ask1Top50q
821328,1589852472000749539,12605445,2000533,94003000,287,28900,29200,28700,28800,524,1633900,47127400000,0,0,0,0,28700,28600,28500,28400,28300,28200,28100,28000,27900,27800,28800,28900,29000,29100,29200,29300,29400,29500,29600,29700,140200,345600,190600,81700,74900,110600,497000,147200,38900,17400,205500,79500,51900,101500,103900,49800,34800,628900,168500,91100,17,38,48,21,18,18,24,30,11,7,47,7,8,15,22,12,11,20,20,18,1200,900,3000,100,5000,3900,84700,500,11800,2500,1000,1500,17500,2300,500,800,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150600,100,9300,100,200,1000,900,13700,5000,400,15000,600,200,100,100,4200,100,100,100,100,100,200,100,100,200,100,100,100,100,100,200,100,100,100,100,200,100,100,300,100,100,100,100,200,200,200,100,0,0,0
823551,1589852475000717700,12644710,2000533,94006000,287,28900,29200,28700,28700,527,1637900,47242200000,0,0,0,0,28700,28600,28500,28400,28300,28200,28100,28000,27900,27800,28800,28900,29000,29100,29200,29300,29400,29500,29600,29700,138200,345600,190600,81700,74900,110600,497000,147200,38900,17400,205100,79500,51900,101500,103900,49800,34800,628900,168500,91100,16,38,48,21,18,18,24,30,11,7,46,7,8,15,22,12,11,20,20,18,1100,100,5000,3900,84700,500,11800,2500,1000,1500,17500,2300,500,800,3000,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150600,100,9300,100,200,1000,900,13700,5000,15000,600,200,100,100,4200,100,100,100,100,100,200,100,100,200,100,100,100,100,100,200,100,100,100,100,200,100,100,300,100,100,100,100,200,200,200,100,0,0,0,0
825783,1589852478000712511,12680477,2000533,94009000,287,28900,29200,28700,28700,527,1637900,47242200000,0,0,0,0,28700,28600,28500,28400,28300,28200,28100,28000,27900,27800,28800,28900,29000,29100,29200,29300,29400,29500,29600,29700,138200,345600,190600,81700,74900,110600,497000,147200,38900,17400,205100,79500,51900,101500,103900,49800,34800,628900,168500,91100,16,38,48,21,18,18,24,30,11,7,46,7,8,15,22,12,11,20,20,18,1100,100,5000,3900,84700,500,11800,2500,1000,1500,17500,2300,500,800,3000,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150600,100,9300,100,200,1000,900,13700,5000,15000,600,200,100,100,4200,100,100,100,100,100,200,100,100,200,100,100,100,100,100,200,100,100,100,100,200,100,100,300,100,100,100,100,200,200,200,100,0,0,0,0
828013,1589852481000714755,12717752,2000533,94012000,287,28900,29200,28700,28700,531,

In [4]:
display(logSZ1["vwapBid"].unique())
display(logSZ1["vwapAsk"].unique())
display(logSZ1["totalBidQuantity"].unique())
display(logSZ1["totalAskQuantity"].unique())

array([0], dtype=int64)

array([0], dtype=int64)

array([0], dtype=int64)

array([0], dtype=int64)

In [8]:
startTm = datetime.datetime.now()
logSZ = pd.read_csv(r'\\192.168.10.30\Kevin_zhenyu\rawData\logs_20200528_zs_92_01_day_data\mdLog_SZ_20200528_0844.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
print(datetime.datetime.now() - startTm)

startTm = datetime.datetime.now()
logSZ1 = pd.read_csv(r'\\192.168.10.30\Kevin_zhenyu\check\20200528\premdgw\mdL2Log_SZ.csv',
                    encoding="utf-8")
display(logSZ1["vwapBid"].unique())
display(logSZ1["vwapAsk"].unique())
display(logSZ1["totalBidQuantity"].unique())
display(logSZ1["totalAskQuantity"].unique())




logSZ1 = logSZ1[["clockAtArrival", "sequenceNo", "ID",
                                              "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "open"]]
print(datetime.datetime.now() - startTm)

print("original data source type: ")
print(logSZ["source"].unique())
print(logSZ["time"].unique())
print(logSZ1["time"].unique())
logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
logSZ1 = logSZ1.rename(columns={"open":"openPrice"})
logSZ1["StockID"] = logSZ1["ID"] - 2000000

array([0], dtype=int64)

array([0], dtype=int64)

array([0], dtype=int64)

array([0], dtype=int64)

0:03:55.441754
original data source type: 
[4 3]
[ 84454000  84457000  84554000 ... 151203000 151303000 151403000]
[ 90042000  90142000  90242000 ... 152330000 152333000 152336000]


AttributeError: 'int' object has no attribute 'replace'

In [13]:
pd.set_option("max_columns", 200)
display(logSZ[(logSZ["StockID"] == 1) & (logSZ["time"] >= 93000000) & (logSZ["source"] == 4)].head(10))
display(logSZ1[(logSZ1["StockID"] == 1) & (logSZ1["time"] >= 93000000)].head(10))

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
291572,1590629420735990,1900366,4,1,SZ,93000000,1115200,14351514.0,12.83,12.82,12.81,12.80,12.79,12.78,21600,72000,91500,3700,53100,12.83,12.84,12.85,12.86,12.87,4000,1900,4000,66900,311700,12.87
294390,1590629423880938,2339748,4,1,SZ,93003000,1462300,18808449.0,12.86,12.85,12.83,12.82,12.81,12.80,27600,13100,40500,33100,172400,12.86,12.87,12.88,12.89,12.90,303578,396500,99050,181300,122700,12.87
298800,1590629426839695,2698750,4,1,SZ,93006000,1616900,20794478.0,12.85,12.84,12.83,12.82,12.81,12.80,300,11100,41600,36000,180500,12.85,12.86,12.87,12.88,12.89,2300,344778,402700,92650,162100,12.87
303199,1590629429824902,2838138,4,1,SZ,93009000,1637700,21061803.0,12.86,12.85,12.84,12.83,12.82,12.81,25000,800,8800,41900,41600,12.86,12.87,12.88,12.89,12.90,348678,401300,98850,102400,136400,12.87
307571,1590629432803054,2927759,4,1,SZ,93012000,1819500,23399515.0,12.85,12.85,12.84,12.83,12.82,12.81,31800,1300,8000,43100,41600,12.86,12.87,12.88,12.89,12.90,198878,405000,98850,102400,137400,12.87
311840,1590629435791931,3004839,4,1,SZ,93015000,2091278,26894859.3,12.85,12.86,12.85,12.84,12.83,12.82,200,18622,3600,6800,43100,12.87,12.88,12.89,12.90,12.91,362300,98850,102400,137400,122600,12.87
315937,1590629438782640,3081190,4,1,SZ,93018000,2107678,27105747.3,12.85,12.85,12.84,12.83,12.82,12.81,214922,3600,6700,43700,47200,12.87,12.88,12.89,12.90,12.91,355700,98850,101200,137400,122600,12.87
320042,1590629441782102,3152998,4,1,SZ,93021000,2202178,28321748.3,12.86,12.85,12.84,12.83,12.82,12.81,239722,3600,6700,43200,47500,12.86,12.87,12.88,12.89,12.90,1100,276900,98650,101200,135400,12.87
324066,1590629444780503,3226344,4,1,SZ,93024000,2240678,28816956.3,12.86,12.85,12.84,12.83,12.82,12.81,232022,3600,6500,43200,47500,12.87,12.88,12.89,12.90,12.91,257600,87650,101200,135400,122600,12.87
328122,1590629447780543,3297042,4,1,SZ,93027000,2270078,29195267.3,12.87,12.86,12.85,12.84,12.83,12.82,13100,216922,3600,7000,48200,12.87,12.88,12.89,12.90,12.91,240900,87350,100600,135400,122600,12.87


,clockAtArrival,sequenceNo,ID,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,StockID
303275,1589851869000596473,2121239,2000001,93000000,1496548,200687646800,134200,134100,134000,133900,133800,133700,109152,618100,49000,82500,83400,134200,134300,134400,134500,134600,62600,45300,54900,55000,33200,134100,1
310609,1589851872000739085,2565525,2000001,93003000,2566338,344343556600,134100,134100,134000,133900,133800,133700,215962,617600,58600,78200,72100,134300,134500,134600,134700,134800,19800,94292,81400,157700,180100,134100,1
313400,1589851875000700024,2982558,2000001,93006000,2798838,375534186600,134200,134100,134000,133900,133800,133700,341162,621300,58600,78200,72100,134200,134300,134400,134500,134600,14500,16200,17500,251592,135300,134100,1
316086,1589851878000685070,3249066,2000001,93009000,2943538,394940936600,134200,134100,134000,133900,133800,133700,222662,621600,58600,78600,72200,134200,134300,134400,134500,134600,3100,470300,20500,260292,141700,134100,1
318787,1589851881000689928,3336758,2000001,93012000,3058338,410340496600,134200,134100,134000,133900,133800,133700,165962,171600,59100,78600,72200,134200,134300,134400,134500,134600,14000,494100,58500,269692,134700,134100,1
321289,1589851884000663287,3402469,2000001,93015000,3262800,437758380800,134100,134000,133900,133800,133700,133600,164400,59100,78600,72200,55100,134100,134200,134300,134400,134500,60938,98300,485000,60000,271692,134100,1
323761,1589851887000665388,3455853,2000001,93018000,3378500,453262670800,134000,134000,133900,133800,133700,133600,53600,59100,79100,72200,55100,134100,134200,134300,134400,134500,167338,234900,477600,66500,271692,134100,1
326157,1589851890000663677,3510445,2000001,93021000,3640700,488399300800,133800,133800,133700,133600,133500,133400,56400,72200,55100,122300,84400,133900,134000,134100,134200,134300,900,225900,77838,280600,477600,134100,1
328527,1589851893000653515,3567824,2000001,93024000,3704500,496935960800,133800,133700,133600,133500,133400,133300,71300,55100,122300,84400,196100,133800,133900,134000,134100,134200,28800,24000,249500,91738,270800,134100,1
330871,1589851896000652204,3625589,2000001,93027000,3718600,498821800800,133700,133700,133600,133500,133400,133300,64000,55300,122300,84400,196100,133800,133900,134000,134100,134200,29000,25300,258100,92738,156100,134100,1


In [16]:
logSZ1[(logSZ1["StockID"] == 1)]["openPrice"].unique()

array([     0, 134100], dtype=int64)

In [17]:
datetime.datetime.fromtimestamp(1589852472000749539/1e9).strftime("%Y-%m-%d %H:%M:%S %f")

'2020-05-19 09:41:12 000750'

In [ ]:
data1 = logSZ[(logSZ["cum_volume"] > 0) & (logSZ["time"] < 145700000) & (logSZ["source"] == 4)]
del logSZ
data2 = logSZ1[(logSZ1["cum_volume"] > 0) & (logSZ1["time"] < 145700000)]
del logSZ1
columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice"]
data1_1["close"] = (data1_1["close"] * 10000).round(0)
data1_1["openPrice"] = (data1_1["openPrice"] * 10000).round(0)
data1_1["bid1p"] = (data1_1["bid1p"] * 10000).round(0)
data1_1["bid2p"] = (data1_1["bid2p"] * 10000).round(0)
data1_1["bid3p"] = (data1_1["bid3p"] * 10000).round(0)
data1_1["bid4p"] = (data1_1["bid4p"] * 10000).round(0)
data1_1["bid5p"] = (data1_1["bid5p"] * 10000).round(0)
data1_1["ask1p"] = (data1_1["ask1p"] * 10000).round(0)
data1_1["ask2p"] = (data1_1["ask2p"] * 10000).round(0)
data1_1["ask3p"] = (data1_1["ask3p"] * 10000).round(0)
data1_1["ask4p"] = (data1_1["ask4p"] * 10000).round(0)
data1_1["ask5p"] = (data1_1["ask5p"] * 10000).round(0)
data1_1["cum_amount"] = (data1_1["cum_amount"] * 10000).round(0)


data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
n1 = len(data1_1["StockID"].unique())
n2 = len(data2_1["StockID"].unique())
print(n1)
print(n2)
if n1 != n2:
    sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
    data1_1 = data1_1[data1_1["StockID"].isin(sl)]
    data2_1 = data2_1[data2_1["StockID"].isin(sl)]
test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["time_x"].count()
n2 = test["time_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["time_y"])])
    print(len(test[np.isnan(test["time_y"])])/n1)
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(test[np.isnan(test["time_x"])])
    display(n2-n1)

### 2. trade data

### 3. order data